In [12]:
from PIL import Image
import imagehash

# Load images
img1 = Image.open('images/1.jpg')
img2 = Image.open('images/7.jpg')

# Compute perceptual hash
hash1 = imagehash.phash(img1)
hash2 = imagehash.phash(img2)

# Compute similarity (Hamming distance)
similarity = 1 - (hash1 - hash2) / len(hash1.hash)**2
print(f"Similarity: {similarity:.2f}")


Similarity: 0.41


In [ ]:
#%pip install imagehash

In [27]:
import cv2
import numpy as np

# Load images
img1 = cv2.imread('images/1.jpg')
img2 = cv2.imread('images/7.jpg')

# Convert to HSV color space
img1_hsv = cv2.cvtColor(img1, cv2.COLOR_BGR2HSV)
img2_hsv = cv2.cvtColor(img2, cv2.COLOR_BGR2HSV)

# Compute histograms
hist1 = cv2.calcHist([img1_hsv], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
hist2 = cv2.calcHist([img2_hsv], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])

# Normalize histograms
cv2.normalize(hist1, hist1)
cv2.normalize(hist2, hist2)

# Compare using correlation
similarity = cv2.compareHist(hist1, hist2, cv2.HISTCMP_CORREL)
print(f"Similarity: {similarity:.2f}")


Similarity: -0.20


In [26]:
from PIL import Image
import imagehash

# Load images
img1 = Image.open('images/1.jpg')
img2 = Image.open('images/7.jpg')

# Compute different hash types
hash_funcs = {
    "aHash": imagehash.average_hash,
    "dHash": imagehash.dhash,
    "pHash": imagehash.phash,
    "wHash": imagehash.whash
}

hash_values1 = {name: func(img1) for name, func in hash_funcs.items()}
hash_values2 = {name: func(img2) for name, func in hash_funcs.items()}

# Compute similarities
similarities = {
    name: 1 - (hash_values1[name] - hash_values2[name]) / len(hash_values1[name].hash)**2
    for name in hash_funcs
}

# Weighted similarity score (adjust weights as needed)
weights = {"aHash": 0.2, "dHash": 0.3, "pHash": 0.4, "wHash": 0.1}
final_similarity = sum(similarities[name] * weights[name] for name in hash_funcs)

# Print results
print("Individual Similarities:", similarities)
print(f"Final Weighted Similarity: {final_similarity:.2f}")


Individual Similarities: {'aHash': 0.578125, 'dHash': 0.390625, 'pHash': 0.40625, 'wHash': 0.625}
Final Weighted Similarity: 0.46


In [ ]:
import cv2
from PIL import Image
import imagehash
import os

# Parameters
video_path = "2024-01-17 21-19-54-done.mkv"  # Change this to your video file
output_dir = "scene_changes"  # Directory to save scene change images
frame_interval = 5  # Check every 5th frame
similarity_threshold = 0.75  # Scene change threshold

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Open video file
cap = cv2.VideoCapture(video_path)

scene_count = 0
last_scene_image = None
last_hash = None

frame_index = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Process only every 5th frame
    if frame_index % frame_interval == 0:
        # Convert frame to PIL image
        pil_img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

        # Compute perceptual hash
        current_hash = imagehash.phash(pil_img)

        # Compare with previous hash
        if last_hash is not None:
            similarity = 1 - (last_hash - current_hash) / len(last_hash.hash) ** 2

            if similarity < similarity_threshold:
                # Scene change detected, save last frame
                if last_scene_image is not None:
                    scene_filename = os.path.join(output_dir, f"scene_{scene_count}.jpg")
                    cv2.imwrite(scene_filename, last_scene_image)
                    print(f"Scene {scene_count} saved: {scene_filename}")

                # Increase scene count
                scene_count += 1

        # Update last scene image and hash
        last_scene_image = frame.copy()
        last_hash = current_hash

    frame_index += 1

cap.release()
print(f"Total scenes detected: {scene_count}")


Scene 0 saved: scene_changes\scene_0.jpg
Scene 1 saved: scene_changes\scene_1.jpg
Scene 2 saved: scene_changes\scene_2.jpg
Scene 3 saved: scene_changes\scene_3.jpg
Scene 4 saved: scene_changes\scene_4.jpg
Scene 5 saved: scene_changes\scene_5.jpg
Scene 6 saved: scene_changes\scene_6.jpg
Scene 7 saved: scene_changes\scene_7.jpg
Scene 8 saved: scene_changes\scene_8.jpg


In [ ]:
import cv2
from PIL import Image
import imagehash
import os

# Parameters
video_path = "2024-01-17 21-19-54-done.mp4"  # Change this to your video file
output_dir = "scene_changes"  # Directory to save scene change images
frame_interval = 5  # Check every 5th frame
similarity_threshold = 0.65  # Scene change threshold

# Hash function weights (adjust if needed)
weights = {"aHash": 0.2, "dHash": 0.3, "pHash": 0.4, "wHash": 0.1}

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Open video file
cap = cv2.VideoCapture(video_path)

scene_count = 0
last_scene_image = None
last_hashes = None

frame_index = 0

# Define hash functions
hash_funcs = {
    "aHash": imagehash.average_hash,
    "dHash": imagehash.dhash,
    "pHash": imagehash.phash,
    "wHash": imagehash.whash
}

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Process only every 5th frame
    if frame_index % frame_interval == 0:
        
        # convert to 720p resolution
        frame = cv2.resize(frame, (1280, 720))

        # Convert frame to PIL image
        pil_img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

        # Compute all hash values
        current_hashes = {name: func(pil_img) for name, func in hash_funcs.items()}

        # Compare with previous frame's hash
        if last_hashes is not None:
            similarities = {
                name: 1 - (last_hashes[name] - current_hashes[name]) / len(last_hashes[name].hash) ** 2
                for name in hash_funcs
            }

            # Compute weighted similarity score
            final_similarity = sum(similarities[name] * weights[name] for name in hash_funcs)

            if final_similarity < similarity_threshold:
                # Scene change detected, save last frame
                if last_scene_image is not None:
                    scene_filename = os.path.join(output_dir, f"scene_{scene_count}.jpg")
                    cv2.imwrite(scene_filename, last_scene_image)
                    print(f"Scene {scene_count} saved: {scene_filename}")

                # Increase scene count
                scene_count += 1

        # Update last scene image and hashes
        last_scene_image = frame.copy()
        last_hashes = current_hashes

    frame_index += 1

cap.release()
print(f"Total scenes detected: {scene_count}")


: 

In [3]:
from PIL import Image
import imagehash
import os

# === CONFIGURABLE ===
folder = r"C:\Users\Krishnaraj\Downloads\testing_scene_detect\detect_histt - Copy"  # TODO: Set your folder
threshold = 5               # Hamming distance threshold (0 = exact match)
# =====================

hashes = {}
to_delete = []  # Store paths of detected duplicates

def is_similar(hash1, hash2, threshold):
    return hash1 - hash2 <= threshold

print(f"Scanning folder: {folder}")
print(f"Using perceptual hash threshold: {threshold}\n")

# Traverse the folder
for filename in os.listdir(folder):
    path = os.path.join(folder, filename)
    if not os.path.isfile(path):
        continue

    try:
        img = Image.open(path)
        img_hash = imagehash.phash(img)

        # Check against all known hashes
        found_duplicate = False
        for h in hashes:
            if is_similar(img_hash, h, threshold):
                print(f"Duplicate detected:")
                print(f" - Original: {hashes[h]}")
                print(f" - Duplicate: {filename} (distance: {img_hash - h})\n")
                to_delete.append(path)
                found_duplicate = True
                break

        if not found_duplicate:
            hashes[img_hash] = filename

    except Exception as e:
        print(f"Error processing {filename}: {e}")

# === Final Summary and Deletion ===
if to_delete:
    print("\nSummary of duplicates detected:")
    for f in to_delete:
        print(f" - {f}")

    choice = input("\nDelete all these duplicate files? (y/n): ").strip().lower()
    if choice == 'y':
        for f in to_delete:
            try:
                os.remove(f)
                print(f"Deleted: {f}")
            except Exception as e:
                print(f"Error deleting {f}: {e}")
    else:
        print("No files were deleted.")
else:
    print("\nNo duplicates detected.")


Scanning folder: C:\Users\Krishnaraj\Downloads\testing_scene_detect\detect_histt - Copy
Using perceptual hash threshold: 5

Duplicate detected:
 - Original: 2024-01-17 21-19-54-done-Scene-001-01.jpg
 - Duplicate: 2024-01-17 21-19-54-done-Scene-001-02.jpg (distance: 2)

Duplicate detected:
 - Original: 2024-01-17 21-19-54-done-Scene-001-01.jpg
 - Duplicate: 2024-01-17 21-19-54-done-Scene-001-03.jpg (distance: 2)

Duplicate detected:
 - Original: 2024-01-17 21-19-54-done-Scene-002-01.jpg
 - Duplicate: 2024-01-17 21-19-54-done-Scene-002-02.jpg (distance: 0)

Duplicate detected:
 - Original: 2024-01-17 21-19-54-done-Scene-002-01.jpg
 - Duplicate: 2024-01-17 21-19-54-done-Scene-002-03.jpg (distance: 0)

Duplicate detected:
 - Original: 2024-01-17 21-19-54-done-Scene-003-01.jpg
 - Duplicate: 2024-01-17 21-19-54-done-Scene-003-02.jpg (distance: 2)

Duplicate detected:
 - Original: 2024-01-17 21-19-54-done-Scene-003-01.jpg
 - Duplicate: 2024-01-17 21-19-54-done-Scene-003-03.jpg (distance: 2)

